In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import math 
import torch
import random
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer

from highlight import *
from highlight import XentLang as X

Loading dataset shards:   0%|          | 0/41 [00:00<?, ?it/s]

In [3]:
model_name = "gpt2-xl"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name, clean_up_tokenization_spaces=True)
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

model.save_pretrained("models/gpt2-xl-M0")

In [8]:
model = AutoModelForCausalLM.from_pretrained("models/gpt2-xl-M0/", local_files_only=True)
tokenizer = AutoTokenizer.from_pretrained("models/gpt2-xl-M0/", local_files_only=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
def generate_data():
    task_manager = HilightWiki()
    textok, t1, t2 = task_manager.extract()

    min_y_window = int(sub_window_size)
    i = random.randint(0, textok.size(-1) - min_y_window)
    y = textok[0][i:i+min_y_window]

    hilisorted = task_manager.xent_sort(y.unsqueeze(0), t1, t2)

    response = detokenize_l(hilisorted)

    keep_best = 20

    red_best = X().comma.join(response[:keep_best])
    blue_best = X().comma.join(response[-keep_best:])

    min_y_window = int(sub_window_size)

    i = random.randint(0, textok.size(-1) - min_y_window)
    slice = textok[0][i:i+min_y_window]
    ywidth = 3
    o = slice[:ywidth]
    c = slice[-ywidth:]

    y = (detokenize_w(o), detokenize_w(c))
    # print(y)
    x1 = detokenize_0(t1)
    x2 = detokenize_0(t2)

    text = detokenize_0(textok)
    func = redblue(y, x1, x2)

    output = text+"\n"+func+"\n"+red_best+X().comma+blue_best

    return output

In [4]:
import pyinstrument

with pyinstrument.profile():
    text = generate_data()



pyinstrument ........................................
.
.  Block at /tmp/ipykernel_3971819/886730326.py:3
.
.  6.603 <module>  ../../../tmp/ipykernel_3971819/886730326.py:1
.  └─ 6.603 generate_data  ../../../tmp/ipykernel_3971819/653696646.py:1
.     ├─ 6.272 HilightWiki.xent_sort  highlight.py:148
.     │  ├─ 5.824 GPT2LMHeadModel._wrapped_call_impl  torch/nn/modules/module.py:1549
.     │  │     [138 frames hidden]  torch, transformers, importlib, <buil...
.     │  │        1.604 _VariableFunctionsClass.embedding  <built-in>
.     │  ├─ 0.235 _VariableFunctionsClass.sort  <built-in>
.     │  └─ 0.211 cross_entropy  torch/nn/functional.py:3014
.     │        [1 frames hidden]  <built-in>
.     ├─ 0.206 HilightWiki.extract  highlight.py:125
.     │  ├─ 0.115 WikiArticle.__init__  highlight.py:80
.     │  │  └─ 0.075 WikiArticle.tokenization  highlight.py:92
.     │  │     └─ 0.075 tokenize  highlight.py:26
.     │  └─ 0.082 WikiArticle.get_training_sample  highlight.py:95
.     │    

In [5]:
# import pstats
# from pstats import SortKey
# import cProfile
# cProfile.run("generate_data()", "restats")
# p = pstats.Stats("restats")
# p.strip_dirs().sort_stats(SortKey.TIME).print_stats()

In [6]:
# gen = 1e5
# separator = "£££$$$£££"

# for i in tqdm(range(int(gen))):
#     output = generate_data()
#     with open("highlighting_data.txt", "a") as f:
#         f.write(output + separator)